In [7]:
pip install tmdbsimple


Note: you may need to restart the kernel to use updated packages.


In [1]:
import tmdbsimple as tmdb
tmdb.API_KEY = 'e4d3e8d00ed7332a43f7ce98098aaf27'

In [2]:
import requests
tmdb.REQUESTS_SESSION = requests.Session()

In [6]:
movie = tmdb.Movies(64)
response = movie.info()
movie.title

'Talk to Her'

In [7]:
 response = movie.releases()
for c in movie.countries:
        if c['iso_3166_1'] == 'US':
             print(c['certification'])




R
R
R
R


In [9]:
search = tmdb.Search()
response = search.movie(query='The Bourne')
for s in search.results:
    print(s['title'], s['id'], s['release_date'], s['popularity'])

The Bourne Identity 2501 2002-06-14 64.391
The Bourne Ultimatum 2503 2007-08-03 62.585
The Bourne Supremacy 2502 2004-07-23 37.359
The Bourne Legacy 49040 2012-08-08 48.241
The Bourne Identity 895795 1988-05-08 4.607
Matthew Bourne's The Red Shoes 748635 2020-09-30 1.004
Matthew Bourne: The Nutcracker 1036829  0.36
Matthew Bourne's The Car Man 702641 2016-03-13 0.84
Bette Bourne: It Goes with the Shoes 179304 2013-03-21 0.593
AVP The Monster Hydro Cup Day 2-1: Men’s Semi-Final 1 - Dalhausser and Lucena vs Tr. Crabb and Bourne 728042 2020-07-19 0.593
Dancing Bournonville 390459 1979-01-01 0.84
Bourn 744344 2020-09-25 0.908


In [10]:
    >>> identity = tmdb.Movies(2501)
    >>> response = identity.info()
    >>> identity.budget, identity.runtime
    (60000000, 119)
    >>> int(identity.budget/identity.runtime)
    504201
    >>> supremacy = tmdb.Movies(2502)
    >>> response = supremacy.info()
    >>> supremacy.budget, supremacy.runtime
    (75000000, 108)
    >>> int(supremacy.budget/supremacy.runtime)
    694444
    >>> ultimatum = tmdb.Movies(2503)
    >>> response = ultimatum.info()
    >>> ultimatum.budget, ultimatum.runtime
    (70000000, 115)
    >>> int(ultimatum.budget/ultimatum.runtime)
    608695

608695

In [12]:
 response = tmdb.Movies(603).info()
 response['budget']


63000000

In [1]:
import sqlite3
import pandas as pd

# List of file paths
file_paths = ['Desktop/Project2/data1.tsv', 'Desktop/Project2/data.tsv', 'Desktop/Project2/title-akas-us-only.csv']

# Connect to SQLite database (or create one if it doesn't exist)
conn = sqlite3.connect('your_database.db')

# Create a cursor object
cursor = conn.cursor()

# Function to load CSV or TSV files into SQLite tables
def load_file_into_sqlite(file_path, table_name, delimiter=','):
    # Read CSV or TSV file into DataFrame
    if delimiter == ',':
        df = pd.read_csv(file_path)
    elif delimiter == '\t':
        df = pd.read_csv(file_path, sep='\t')
    else:
        raise ValueError("Delimiter must be ',' or '\t'")
    
    # Write DataFrame to SQLite table
    df.to_sql(table_name, conn, index=False, if_exists='replace')

# Load CSV files into SQLite tables
csv_files = [('Desktop/Project2/title-akas-us-only.csv', 'table1')]  

for csv_file, table_name in csv_files:
    load_file_into_sqlite(csv_file, table_name)

# Load TSV files into SQLite tables
tsv_files = [('Desktop/Project2/data1.tsv', 'table2'), ('Desktop/Project2/data.tsv', 'table3')]  

for tsv_file, table_name in tsv_files:
    load_file_into_sqlite(tsv_file, table_name, delimiter='\t')

C:\Users\asus\AppData\Local\Temp\ipykernel_31804\3874403701.py:17: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\asus\AppData\Local\Temp\ipykernel_31804\3874403701.py:19: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t')


In [20]:
import tmdbsimple as tmdb
import pandas as pd
import gzip
import os

# Set your TMDb API key
tmdb.API_KEY = 'e4d3e8d00ed7332a43f7ce98098aaf27'

# Function to fetch movie data
# Function to fetch movie data
def fetch_movie_data(start_year, end_year):
    all_movies = []
    for year in range(start_year, end_year + 1):
        page = 1
        while True:
            response = tmdb.Discover().movie(page=page, certification_country='US', certification_lte='R', sort_by='popularity.desc', release_date_gte=f'{year}-01-01', release_date_lte=f'{year}-12-31')
            if 'results' in response:
                for movie in response['results']:
                    movie_info = extract_movie_info(movie)
                    movie_info['Release_Year'] = year  # Add release year to movie info
                    all_movies.append(movie_info)
                if page < response['total_pages']:
                    page += 1
                else:
                    break
            else:
                break
    return all_movies

# Function to extract required information from movie data
def extract_movie_info(movie):
    title = movie.get('title', None)
    budget = movie.get('budget', None)
    revenue = movie.get('revenue', None)
    certification = movie.get('release_dates', {}).get('results', [{}])[0].get('certification', None)
    return {
        'Title': title,
        'Budget': budget,
        'Revenue': revenue,
        'Certification': certification
    }

# Main function to fetch and process movie data
def main(start_year, end_year):
    movie_data = fetch_movie_data(start_year, end_year)
    for year in range(start_year, end_year + 1):
        year_movies = [movie for movie in movie_data if movie['Release_Year'] == year]
        df = pd.DataFrame(year_movies)
        file_name = f'movie_data_{year}.csv.gz'
        with gzip.open(file_name, 'wt', encoding='utf-8') as f:
            df.to_csv(f, index=False)
        print(f'Movie data for {year} saved as {file_name}!')

# Specify the range of years
start_year = 2001
end_year = 2002

# Fetch and process movie data
main(start_year, end_year)

Movie data for 2001 saved as movie_data_2001.csv.gz!
Movie data for 2002 saved as movie_data_2002.csv.gz!


In [21]:
def fetch_movie_data(tmdb_id):
    try:
        movie = tmdb.Movies(tmdb_id)
        response = movie.info()
        # Extract relevant information from the response
        title = response.get('title', None)
        budget = response.get('budget', None)
        revenue = response.get('revenue', None)
        certification = response.get('release_dates', {}).get('results', [{}])[0].get('certification', None)
        return {
            'Title': title,
            'Budget': budget,
            'Revenue': revenue,
            'Certification': certification
        }
    except Exception as e:
        print(f"Error fetching movie data for TMDb ID {tmdb_id}: {e}")
        return None

# Test the function for "The Avengers" and "The Notebook" TMDb IDs
avengers_id = 24428  # TMDb ID for The Avengers
notebook_id = 11036  # TMDb ID for The Notebook

avengers_data = fetch_movie_data(avengers_id)
notebook_data = fetch_movie_data(notebook_id)

print("The Avengers data:", avengers_data)
print("The Notebook data:", notebook_data)

The Avengers data: {'Title': 'The Avengers', 'Budget': 220000000, 'Revenue': 1518815515, 'Certification': None}
The Notebook data: {'Title': 'The Notebook', 'Budget': 29000000, 'Revenue': 115600000, 'Certification': None}


In [29]:
## make a movie object using the .Movies function from tmdb
movie = tmdb.Movies(603)




In [30]:
info = movie.info()
print(info)

{'adult': False, 'backdrop_path': '/icmmSD4vTTDKOq2vvdulafOGw93.jpg', 'belongs_to_collection': {'id': 2344, 'name': 'The Matrix Collection', 'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg', 'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'}, 'budget': 63000000, 'genres': [{'id': 28, 'name': 'Action'}, {'id': 878, 'name': 'Science Fiction'}], 'homepage': 'http://www.warnerbros.com/matrix', 'id': 603, 'imdb_id': 'tt0133093', 'origin_country': ['US'], 'original_language': 'en', 'original_title': 'The Matrix', 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.', 'popularity': 480.776, 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg', 'production_companies': [{'id': 79, 'logo_path': '/at4uYdwAAgNRKhZuuFX8ShKSybw.png', 'name': 'Village Roadshow Pictures', 'origin_country': 'US'}, {'id': 372, 'logo_path': None, 'name': 'Groucho II Film Partnershi

In [31]:
info['budget']

63000000

In [32]:
info['revenue']

463517383

In [33]:
info['imdb_id']

'tt0133093'

In [34]:
# example from package README
# source = https://github.com/celiao/tmdbsimple
releases = movie.releases()
for c in releases['countries']:
    if c['iso_3166_1'] == 'US':
        print(c['certification'])

R
R


In [36]:
df = pd.read_csv('Desktop/Project2/data1.tsv', sep='\t')


C:\Users\asus\AppData\Local\Temp\ipykernel_31804\2434500045.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Desktop/Project2/data1.tsv', sep='\t')


In [37]:
df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [39]:
# Define the range of years you wish to retrieve
start_year = 2000
end_year = 2020  # Adjust this to the current year or any other end year you prefer

# Create a list of years from start_year to end_year (inclusive)
years_list = list(range(start_year, end_year + 1))

# Print the list of years
print(years_list)

[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]


In [40]:
from tqdm.notebook import tqdm

# List of years
years_list = [2000, 2001, 2002, 2003]  # Example list of years

# Outer loop with progress bar
for year in tqdm(years_list, desc="Processing years"):
    # Define JSON filename with the year
    json_filename = f"data_{year}.json"

    # Your code to process data for the current year and save results to json_filename
    # Example: fetch data for the current year and save it to json_filename
    # fetch_data_for_year(year)
    # save_results_to_json(results, json_filename)

    # Print progress information
    print(f"Processing data for year {year} and saving to {json_filename}")

Processing years:   0%|          | 0/4 [00:00<?, ?it/s]

Processing data for year 2000 and saving to data_2000.json
Processing data for year 2001 and saving to data_2001.json
Processing data for year 2002 and saving to data_2002.json
Processing data for year 2003 and saving to data_2003.json


In [52]:
import os
import json
from tqdm import tqdm

# Outer loop with progress bar
for year in tqdm(years_list, desc="Processing years"):
    # Define JSON filename with the year
    json_filename = f"data_{year}.json"

    # Check if the JSON file exists
    if not os.path.exists(json_filename):
        # If it doesn't exist, create an empty JSON file
        with open(json_filename, 'w') as f:
            # Write an empty dictionary
            json.dump({}, f)
        print(f"Created JSON file for year {year}: {json_filename}")
    else:
        # If it exists, print a message
        print(f"JSON file for year {year} already exists: {json_filename}")

    # Your code to process data for the current year and save results to json_filename
    # Example: fetch data for the current year and save it to json_filename
    # fetch_data_for_year(year)
    # save_results_to_json(results, json_filename)

    # Print progress information
    print(f"Processing data for year {year} and saving to {json_filename}")




Processing years: 100%|████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 4004.11it/s]

JSON file for year 2000 already exists: data_2000.json
Processing data for year 2000 and saving to data_2000.json
JSON file for year 2001 already exists: data_2001.json
Processing data for year 2001 and saving to data_2001.json
JSON file for year 2002 already exists: data_2002.json
Processing data for year 2002 and saving to data_2002.json
JSON file for year 2003 already exists: data_2003.json
Processing data for year 2003 and saving to data_2003.json


In [56]:
import pandas as pd
import os

# Assuming you have loaded the IMDb title basics data into a DataFrame named df
# and you have a list of years named years_list

# Outer loop with progress bar
for year in tqdm(years_list, desc="Processing years"):
    # Define JSON filename with the year
    json_filename = f"data_{year}.json"

    # Check if the JSON file exists
    if os.path.exists(json_filename):
        try:
            # Load the contents of the JSON file
            with open(json_filename, 'r') as file:
                json_data = file.read()
                if json_data.strip():  # Check if JSON data is not empty
                    json_data = pd.read_json(json_data, orient='records')
                    # Filter the data for the selected year
                    selected_year_data = df[df['startYear'] == year]

                    # Extract movie IDs for the selected year
                    movie_ids = selected_year_data[selected_year_data['titleType'] == 'movie']['tconst'].tolist()

                    # Check for previously downloaded movie IDs and filter out duplicates
                    existing_movie_ids = json_data['imdb_id'].tolist()
                    movie_ids_to_get = [movie_id for movie_id in movie_ids if movie_id not in existing_movie_ids]

                    # Save the final list of movie_ids_to_get without duplicates
                    print("Final list of movie IDs to get:", movie_ids_to_get)
                else:
                    print(f"JSON file for year {year} is empty.")
        except Exception as e:
            print(f"Error reading JSON file for year {year}: {e}")
    else:
        print(f"JSON file for year {year} does not exist.")




Processing years: 100%|████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 1332.79it/s]

JSON file for year 2000 is empty.
Error reading JSON file for year 2001: If using all scalar values, you must pass an index
Error reading JSON file for year 2002: If using all scalar values, you must pass an index
Error reading JSON file for year 2003: If using all scalar values, you must pass an index


In [57]:
import json
import os

# Function to make API calls and extract results
def fetch_results(movie_id):
    # Your code to fetch results for a given movie ID
    pass

# Outer loop with progress bar
for year in tqdm(years_list, desc="Processing years"):
    # Define JSON filename with the year
    json_filename = f"data_{year}.json"

    # Check if the JSON file exists
    if os.path.exists(json_filename):
        try:
            # Load the contents of the JSON file
            with open(json_filename, 'r') as file:
                json_data = json.load(file)
                
            # Filter the data for the selected year
            selected_year_data = df[df['startYear'] == year]

            # Extract movie IDs for the selected year
            movie_ids = selected_year_data[selected_year_data['titleType'] == 'movie']['tconst'].tolist()

            # Check for previously downloaded movie IDs and filter out duplicates
            existing_movie_ids = [entry['imdb_id'] for entry in json_data]
            movie_ids_to_get = [movie_id for movie_id in movie_ids if movie_id not in existing_movie_ids]

            # Inner loop for making API calls for each movie ID to get results
            for movie_id in movie_ids_to_get:
                # Make API call and extract results
                results = fetch_results(movie_id)
                
                # Append new results to the list from the JSON file
                json_data.append(results)

            # Save the updated JSON file back to the disk
            with open(json_filename, 'w') as file:
                json.dump(json_data, file, indent=4)

        except Exception as e:
            print(f"Error processing JSON file for year {year}: {e}")

    else:
        print(f"JSON file for year {year} does not exist.")


Processing years:   0%|                                                                          | 0/4 [00:00<?, ?it/s]

Error processing JSON file for year 2000: Expecting value: line 1 column 1 (char 0)


Processing years:  50%|█████████████████████████████████                                 | 2/4 [00:00<00:00,  2.78it/s]

Error processing JSON file for year 2001: string indices must be integers, not 'str'


Processing years:  75%|█████████████████████████████████████████████████▌                | 3/4 [00:01<00:00,  1.98it/s]

Error processing JSON file for year 2002: string indices must be integers, not 'str'


Processing years: 100%|██████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.87it/s]

Error processing JSON file for year 2003: string indices must be integers, not 'str'
